<p>
  <a href="https://colab.research.google.com/github/neo4j-partners/hands-on-lab-neo4j-and-vertex-ai/blob/main/Lab%202%20-%20Moving%20Data/playing-around.ipynb" target="_blank">
    <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> Run in Colab
  </a>
</p>

First off, you'll also need to install a few packages.

In [ ]:
!pip install --quiet --upgrade neo4j
!pip install --quiet google-cloud-storage

You'll need to enter the credentials from your AuraDS instance below. You can get your credentials by following this walkthrough.

The "DB_NAME" is always neo4j for AuraDS. It is different from the name you gave your database tenant in the AuraDS console.

In [ ]:
DB_URL = "neo4j+s://XXXXX.databases.neo4j.io"
DB_USER = "neo4j"
DB_PASS = "YOUR PASSWORD"
DB_NAME = "neo4j"

In [ ]:
import pandas as pd
from neo4j import GraphDatabase


driver = GraphDatabase.driver(DB_URL, auth=(DB_USER, DB_PASS))


First we're going to create an in memory graph represtation of the data in Neo4j Graph Data Science (GDS).

Note, if you get an error saying the graph already exists, that's probably because you ran this code before. You can destroy it using the command in the cleanup section of this notebook.

In [ ]:
with driver.session(database=DB_NAME) as session:
    result = session.read_transaction(
        lambda tx: tx.run(
            """
    CALL gds.graph.create.cypher('client_graph', 
      'MATCH (c:Client) RETURN id(c) as id, c.num_transactions as num_transactions, c.total_transaction_amnt as total_transaction_amnt, c.is_fraudster as is_fraudster',
      'MATCH (c:Client)-[:PERFORMED]->(t:Transaction)-[:TO]->(c2:Client) return id(c) as source, id(c2) as target, sum(t.amount) as amount, "TRANSACTED_WITH" as type ')
    """
        ).data()
    )
df = pd.DataFrame(result)
display(df)

Now we can generate an embedding from that graph. This is a new feature we can use in our predictions. We're using FastRP, which is a more full featured and higher performance of Node2Vec. You can learn more about that [here](https://neo4j.com/docs/graph-data-science/current/algorithms/fastrp/).

In [ ]:
with driver.session(database=DB_NAME) as session:
    result = session.read_transaction(
        lambda tx: tx.run(
            """
    CALL gds.graph.streamNodeProperties
    ('client_graph', ['embedding', 'num_transactions', 'total_transaction_amnt', 'is_fraudster'])
    YIELD nodeId, nodeProperty, propertyValue
    RETURN nodeId, nodeProperty, propertyValue
    """
        ).data()
    )
df = pd.DataFrame(result)
df.head()

Now we need to take that dataframe and shape it into something that better represents our classification problem.

In [ ]:
x = df.pivot(index="nodeId", columns="nodeProperty", values="propertyValue")
x = x.reset_index()
x.columns.name = None
x.head()

Note that the embedding row is an array. To make this dataset more consumable, we should flatten that out into multiple individual features: embedding_0, embedding_1, ... embedding_n.

In [ ]:
FEATURES_FILENAME = "features.csv"

embeddings = pd.DataFrame(x["embedding"].values.tolist()).add_prefix("embedding_")
merged = x.drop(columns=["embedding"]).merge(
    embeddings, left_index=True, right_index=True
)
features_df = merged.drop(
    columns=["is_fraudster", "num_transactions", "total_transaction_amnt"]
)
train_df = merged.drop(columns=["nodeId"])

features_df.to_csv(FEATURES_FILENAME, index=False)